In [ ]:
# imports

import paths
import processing_parameters
import cv2
import pandas as pd
import numpy as np
import functions_bondjango as bd
import functions_loaders as fl
import functions_io as fi
import functions_vame as fv
import importlib
import os

import panel as pn
import holoviews as hv
from holoviews import opts, dim
hv.extension('bokeh')

In [ ]:
# load the preproc files

importlib.reload(processing_parameters)
importlib.reload(fl)

# get the paths from the database using search_list
data_all = bd.query_database('analyzed_data', processing_parameters.vame_video)
data_all = [el for el in data_all if '_preproc' in el['slug']][0]
data_path = data_all['analysis_path']
data_vame_name = data_all['slug'].replace('_preprocessing', '')


data, _, meta_list  = fl.load_preprocessing([data_path], [data_all], behavior_flag=True)
data = data[0]
meta_list = meta_list[0]

real_corners = pd.read_hdf(data_path, 'arena_corners')
dlc_corners = pd.read_hdf(data_path, 'corners')
frame_bounds = pd.read_hdf(data_path, 'frame_bounds')

print(frame_bounds, meta_list, data.shape)

In [ ]:
%%time
# load the corresponding video

# assemble the path
video_path = os.path.join(paths.videoexperiment_path,
                          data_all['slug'].replace('_preprocessing', '.avi'))
print(video_path)
# create the video object
cap = cv2.VideoCapture(video_path)
# allocate memory for the corners
frame_list = []
# # define sigma for the edge detection parameters
# sigma = 0.2
# get the frames to mode
for frames in np.arange(frame_bounds.loc[0, 'end']):

    # read the image
    frame_list.append(cap.read()[1])

# release the capture
cap.release()
frame_list = frame_list[frame_bounds.loc[0, 'start']:]

print(f'Frames after trimming bounds: {len(frame_list)}')

In [ ]:
# create a distortion corrected video

# define the path for saving the movies
temp_path = paths.temp_path
# clean the folder
fi.delete_contents(temp_path)

# get the reference corners 
ref_corners = paths.arena_coordinates['miniscope']

# create a bounded movie to align later
# assemble the bounded movie path
bounded_path = os.path.join(temp_path, 'bounded.avi')
# new_mat = cv2.UMat(rot_matrix)

# save the bounded movie
# get the width and height
width = frame_list[0].shape[1]
height = frame_list[0].shape[0]

# test = cv2.warpPerspective(frames_formotif[0].astype('float32'), rot_matrix.to_numpy(), (width, height))
# current_matrix = rot_matrix.to_numpy()

perspective_matrix = cv2.getPerspectiveTransform(dlc_corners.to_numpy().T.astype('float32'),
                                                 (np.array(ref_corners).astype('float32')+5)*20)

# create the writer
out = cv2.VideoWriter(bounded_path,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (1280, 1024))
# save the movie
for frames in frame_list:
    # apply the perspective matrix
    out_frame = cv2.warpPerspective(frames.astype('float32'), perspective_matrix, (1280, 1024))
    out.write(out_frame.astype('uint8'))

out.release()

In [ ]:
print(coordinate_columns, data.columns)
print(ego_traces)
hv.Raster(ego_traces).opts(tools=['hover'])

In [ ]:
# align the video egocentrically
importlib.reload(fv)

# create the egocentric movie
# path_dlc = data_path
# path_vame = target_folder
file_format = '.avi'
crop_size = (200, 200)
use_video = True
check_video = False
save_align = False

# select the coordinate columns
coordinate_columns = [el for el in data.columns if ('mouse_' in el) | ('cricket_' in el)]

scaled_data = (data[coordinate_columns].fillna(0)+5)*20/1000
print(scaled_data.shape)

ego_traces, egocentric_frames, columns_out = fv.align_demo(scaled_data, [], data_vame_name, file_format, crop_size,
                                   use_video=use_video, check_video=check_video, 
                                   vid_path=bounded_path)

# turn the list into an array
egocentric_frames = np.array(egocentric_frames)
print(egocentric_frames.shape)

In [ ]:
# create the egocentric movie

# assemble the bounded movie path
egocentric_path = os.path.join(temp_path, 'egocentric.avi')

# save the bounded movie
# get the width and height
width = egocentric_frames[0].shape[1]
height = egocentric_frames[0].shape[0]
print(egocentric_frames[0].shape)

# create the writer
out2 = cv2.VideoWriter(egocentric_path,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (height, width))
# save the movie
for frames in egocentric_frames:
    out_frame = np.repeat(np.expand_dims(frames, 2), 3, axis=2)
    out2.write(out_frame.astype('uint8'))

out2.release()

In [ ]:
# hv.Raster(out_frame[:, :, 10])
hv.Raster(egocentric_frames[10, :, :]).opts(tools=['hover'])

In [ ]:
print(type(egocentric_frames))

In [ ]:
# Get the motif locations

# get the present motifs
motif_vector = data['motifs'].to_numpy()
present_motifs = np.unique(motif_vector)
present_motifs = present_motifs[~np.isnan(present_motifs)]

# allocate memory for all the locations
location_perfile = []
duration_perfile = []
# for all the motifs
for motif in present_motifs:
    
    # find all the starts and ends for this motif
    m_idx = (current_motifs==motif).astype(int)
    starts = np.argwhere(np.diff(np.pad(m_idx, (1, 1), mode='constant', constant_values=(0, 0)))==1)
    ends = np.argwhere(np.diff(np.pad(m_idx, (1, 1), mode='constant', constant_values=(0, 0)))==-1)

    # skip if any of the arrays is empty
    if (starts.shape[0] == 0) or (ends.shape[0] == 0):
        duration_perfile.append(np.empty((0, 1)))
        location_perfile.append(np.empty((0, 1)))
        continue
    # trim the starts and ends based on ordering
    if starts[0] > ends[0]:
        if ends.shape[0] > 1:
            ends = ends[1:]
        else:
            duration_perfile.append(np.empty((0, 1)))
            location_perfile.append(np.empty((0, 1)))
            continue
    if starts[-1] > ends [-1]:
        if starts.shape[0] > 1:
            starts = starts[:-1]
        else:
            duration_perfile.append(np.empty((0, 1))) 
            location_perfile.append(np.empty((0, 1)))
            continue
    # trim the starts or ends depending on size
    if starts.shape[0] > ends.shape[0]:
        starts = starts[:-1]
    if ends.shape[0] > starts.shape[0]:
        ends = ends[1:]
    # make sure the ends are always bigger than the starts
    try: 
        assert np.all((ends-starts)>0) 
    except AssertionError:
        print(str(idx)+'_'+str(motif))
        print(starts)
        print(ends)

    # save the locations for this motif
    location_perfile.append([el[0] for el in starts])
    duration_perfile.append([el[0] for el in ends-starts])
        
print(location_perfile)
print(duration_perfile)
print(motif_vector)
# print(location_perfile)
# print(duration_perfile)

In [ ]:
# Create egocentric movies for all motifs

# for all the motifs present
for idx, motif in enumerate(present_motifs):
    # get the maximum duration
    max_duration = np.max(duration_perfile[idx])
    # get the start of the maximum duration
    max_location = location_perfile[idx][np.argmax(duration_perfile[idx])]

    # get the video frames
    frame_idx = np.array(np.arange(max_location, max_location+max_duration))
    motif_frames = egocentric_frames[frame_idx]
    # save the movie
    
    # assemble the bounded movie path
    motif_path = os.path.join(temp_path, str(motif)+'_motif.avi')
    # create the writer
    out2 = cv2.VideoWriter(motif_path,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (200,200))
    # save the movie
    for frames in motif_frames:
        out_frame = np.repeat(np.expand_dims(frames, 2), 3, axis=2)
        out2.write(out_frame.astype('uint8'))

    out2.release()